In [6]:
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import io
import asyncio

from telegram.ext import ApplicationBuilder, MessageHandler, filters
from telegram import Update
from telegram.ext import ContextTypes
import nest_asyncio
nest_asyncio.apply()
import os
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2
from telegram import Update
from telegram.ext import (
    ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
)

In [3]:
!pip install nest_asyncio

In [16]:
!pip install python-telegram-bot --upgrade
!pip install transformers
!pip install torch
!pip install Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 11.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 11.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [2]:
import os
import logging
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
from telegram import Update
from telegram.ext import Application, MessageHandler, filters, ContextTypes

# --- 1. Задаем токен ---
TELEGRAM_TOKEN = "7637994873:AAFagLArVj1RyAR1KKIBvKnHqAqRJaNASHE"

# --- 2. Грузим модель и процессор (это занимает время, можно вынести отдельно) ---
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# --- 3. Настраиваем логгирование (по желанию) ---
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)

# --- 4. Функция обработки фото ---
async def handle_photo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    photo = update.message.photo[-1]
    photo_file = await photo.get_file()
    image_path = "received.jpg"
    await photo_file.download_to_drive(image_path)
    
    # Открываем картинку
    image = Image.open(image_path).convert("RGB")
    
    # --- OCR ---
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # --- Приводим первую букву к верхнему регистру (как хочешь) ---
    if text:
        text = text.strip()
        text = text.capitalize()
    else:
        text = "Не удалось распознать текст. Проверь качество фото."

    await update.message.reply_text(f"Распознанный текст: {text}")

    # Чистим за собой
    os.remove(image_path)

/opt/anaconda3/envs/tf_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.hf.co\', port=443): Max retries exceeded with url: /microsoft/trocr-base-handwritten/25a40cddc7e6120140a3d5b9e3dd3878a92ada7b4f312953ab22edc19c2a5acc?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1748671905&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0ODY3MTkwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9taWNyb3NvZnQvdHJvY3ItYmFzZS1oYW5kd3JpdHRlbi8yNWE0MGNkZGM3ZTYxMjAxNDBhM2Q1YjllM2RkMzg3OGE5MmFkYTdiNGYzMTI5NTNhYjIyZWRjMTljMmE1YWNjP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=n9HmcypPtT4vBARgL6F78GDca3RzE0e1AwnJ9-sVAb~a3STjKBU93nv84HAT8lfGVi7d9GQYmCTmCfG6xMgTog5rvxkjK6KMJer7KdOUPZr4obswh0MDiahjzw5y9ReA~Tzcq5U8YmoeJ3mSef0TJV0bSGpsyWr1msxVToa3UblZFc8gKij0JnUbfGUZYmPGZ67UE9v4GhztzlrQl3jM0BTMoAk7FjttOLUxD3hJIBdJ4nH6aXjzZlqumEj51N462oCR8mfywDBp2pQaBtiOILQZL14JNSS~QGnN~5ZDSgF5DFpa

In [ ]:
# --- 5. Main ---
def main():
    app = Application.builder().token(TELEGRAM_TOKEN).build()
    app.add_handler(MessageHandler(filters.PHOTO, handle_photo))
    print("Бот запущен. Жду фоточек...")
    app.run_polling()

if __name__ == "__main__":
    main()


In [16]:
import os
print(os.getcwd())


/Users/elmansulejmanov/Desktop/Задача №1 Сортировка токсичных комментариев


In [20]:
import os
print(os.getcwd())  # убедись, что тут "/Users/elmansulejmanov/Desktop/Задача №1 Сортировка токсичных комментариев"
print(os.listdir('.'))  # выведет все файлы в этой папке

/Users/elmansulejmanov/Desktop/Задача №1 Сортировка токсичных комментариев
['housing.csv', 'tensorflow_checklist.ipynb', 'adult.csv', 'all_letters_info.csv', 'gym_churn.csv', 'Mall_Customers.csv', 'best_model.h5', '.DS_Store', '3 Задачи matplotlib.ipynb', 'StudentsPerformance.csv', 'labeled.csv', 'Курсы по Machine Learning Engineering .ipynb', 'test.csv', '1 Pandas.ipynb', 'heart.csv', 'submission_blend3_weighted_047.csv', 'test.json', 'Telegram bot.ipynb', 'Задача Kaggle #2', 'Мой первый проект.ipynb', 'Задача2.ipynb', 'model.keras', '2 matlplotlib.ipynb', 'tmp.csv', 'train.json', '4 Машинное обучение sklearn.ipynb', 'train.csv', '.ipynb_checkpoints', 'ЗАДАЧА1.ipynb', '6 Создание нейросетей TenserFlow.ipynb', 'Тренировка №1.ipynb', 'sosi.ipynb', 'submission_catboost1.csv', 'Задача 1 Kaggle Score 0.80617.ipynb', 'catboost_info', 'sample_submission.csv', '5 Задачи Sklearn.ipynb']


In [22]:
model = tf.keras.models.load_model('model.keras')

ValueError: File not found: filepath=model.keras. Please ensure the file is an accessible `.keras` zip file.

In [27]:
import torch
print(torch.__version__)


2.7.0


In [8]:
import requests
print(requests.get('https://google.com').status_code)


200
